#### RNN forward networks

In [1]:
import tensorflow as tf

num_cell = 128

hidden_units = 30

def rnn_forward(inputs):
    # Define the cell, can be either LSTM or GRU, or the peephole implementation.
    cell = tf.contrib.rnn.BasicLSTMCell(num_cell)
    
    # Here we define a dynamic rnn. It builds a dynamic graph such that every time we can pass a batch with varied lengths.
    # time_major=False means the inputs must be [batch_size, time_step, features], otherwise [time_step, batch_size, features]
    # sequence_length is not necessarily to be specified.
    rnn_outputs, states = tf.nn.dynamic_rnn(cell=cell,
                                            inputs=inputs,
                                            sequence_length=None,
                                            dtype=tf.float32,
                                            time_major=False)
    
    
  
    # Logistic layers, choose the last time step as inputs
    h_inputs = rnn_outputs[:, -1, :]
    
    before_outputs = tf.contrib.layers.fully_connected(h_inputs, hidden_units, activation_fn=tf.nn.relu)
    outputs = tf.contrib.layers.fully_connected(before_outputs, 1, activation_fn=None)
    return outputs

In [6]:
with tf.name_scope("asd"):
  logits = rnn_forward(embedded)
tf.trainable_variables()


ValueError: Attempt to have a second RNNCell use the weights of a variable scope that already has weights: 'rnn/basic_lstm_cell'; and the cell was not constructed as BasicLSTMCell(..., reuse=True).  To share the weights of an RNNCell, simply reuse it in your second calculation, or create a new one with the argument reuse=True.

#### Read IMDB dataset using Keras API

In [2]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import numpy as np

(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=10000,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)

def generate_dataset(x_train, y_train, batch_num):
    indices = np.arange(y_train.shape[0])
    np.random.shuffle(indices)
    x_train, y_train = x_train[indices], y_train[indices]
    xs = np.split(x_train, batch_num)
    ys = np.split(y_train, batch_num)
    return xs, ys

  
def max_len(sentences):
    sequence_length = max(len(x) for x in sentences)
    return sequence_length

Using TensorFlow backend.


#### Build graph

In [3]:
X = tf.placeholder(name='inputs', shape=[None, None], dtype=tf.int32)
y = tf.placeholder(name='labels', shape=[None, 1], dtype=tf.float32)

# Embedding matrix
num_words = 10000
embedding_size = 128
embedding = tf.get_variable('Embedding', shape=[num_words, embedding_size], initializer=tf.random_uniform_initializer(-1., 1.))

# Convert tokens into word vectors
embedded = tf.nn.embedding_lookup(embedding, X)

logits = rnn_forward(embedded)

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits))

probs = tf.sigmoid(logits)

train_op = tf.train.AdamOptimizer(0.01).minimize(loss)

#### Training

In [4]:
num_epochs = 3
batch_size = 100
batch_num = y_train.shape[0] / batch_size

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for k in range(num_epochs):
        xs, ys = generate_dataset(x_train, y_train, batch_num)
        for j in range(batch_num):
            
            # We can pass different time step at every iteration of dynamic RNN
            maxlen = max_len(xs[j])
            if maxlen > 100:
                maxlen = 100
                
            X_feed = sequence.pad_sequences(xs[j], maxlen=maxlen)
            y_feed = ys[j].reshape(batch_size, 1)
            loss_val, _ , probs_val = sess.run([loss, train_op, probs], feed_dict={X: X_feed, y: y_feed})
            
            if j % 50 == 0:
                predictions = [1 if i > 0.5 else 0 for i in probs_val]
                accurate_pred = predictions == ys[j]
                print "Epoch %d, step %d " % (k, j)
                print "Train loss and accuracy ", loss_val, float(sum(accurate_pred)) / batch_size, "\n"
                
    # Test performance
    test_xs, test_ys = generate_dataset(x_test, y_test, batch_num)
    acc_count = []
    for j in range(batch_num):
    # We can pass different time step at every iteration of dynamic RNN
        maxlen = max_len(test_xs[j])
        if maxlen > 100:
            maxlen = 100
                
        X_feed = sequence.pad_sequences(test_xs[j], maxlen=maxlen)
        y_feed = test_ys[j].reshape(batch_size, 1)
        probs_val = sess.run(probs, feed_dict={X: X_feed, y: y_feed})
        predictions = [1 if i > 0.5 else 0 for i in probs_val]
        accurate_pred = predictions == test_ys[j]
        acc_count.append(sum(accurate_pred))
    
    print "Test accuracy ", float(sum(acc_count)) / y_test.shape[0], "\n"

Epoch 0, step 0 
Train loss and accuracy  0.6944 0.49 

Epoch 0, step 50 
Train loss and accuracy  0.579625 0.7 

Epoch 0, step 100 
Train loss and accuracy  0.51301 0.76 

Epoch 0, step 150 
Train loss and accuracy  0.368837 0.83 

Epoch 0, step 200 
Train loss and accuracy  0.409801 0.79 

Epoch 1, step 0 
Train loss and accuracy  0.309959 0.87 

Epoch 1, step 50 
Train loss and accuracy  0.275962 0.89 

Epoch 1, step 100 
Train loss and accuracy  0.296119 0.9 

Epoch 1, step 150 
Train loss and accuracy  0.292611 0.9 

Epoch 1, step 200 
Train loss and accuracy  0.17538 0.94 

Epoch 2, step 0 
Train loss and accuracy  0.146157 0.96 

Epoch 2, step 50 
Train loss and accuracy  0.266658 0.88 

Epoch 2, step 100 
Train loss and accuracy  0.226123 0.89 

Epoch 2, step 150 
Train loss and accuracy  0.245804 0.9 

Epoch 2, step 200 
Train loss and accuracy  0.229435 0.88 

Test accuracy  0.8518 

